In [1]:
# ==================== ONE-CELL EVALUATION (SELF-CONTAINED / BOOTSTRAP) ====================
# Kjo qelizë mund të ekzekutohet vetëm, pa varësi nga qelizat e tjera.
# Bën bootstrap të pikeve, distanceve, BF/GA/SA nëse mungojnë; pastaj bën vlerësimin 30-run.

import os, time, math, itertools, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ---- Reproducibility seeds (global) ----
random.seed(42)
np.random.seed(42)


# ---- 0) Bootstrap tourist points (INPUT si LISTË e vetme) ----
import ast
# --- NEW ---
import json, os

def _load_spots_from_file(tag, default_spots):
    """
    Lexon listën nga L1 / L2 / L3 (pa ose me prapashtesë .json/.txt).
    Formate të lejuara:
      - JSON: [ ["Emri", [lat, lon]], ... ]
      - Python literal: [ ("Emri", (lat, lon)), ... ]
    """
    candidates = [tag, f"{tag}.json", f"{tag}.txt"]
    for path in candidates:
        if not os.path.exists(path):
            continue
        try:
            # Provo JSON
            if path.endswith(".json") or path in {"L1","L2","L3"}:
                with open(path if path.endswith(".json") else f"{path}.json", "r", encoding="utf-8") as f:
                    data = json.load(f)
                spots = []
                for name, coords in data:
                    lat, lon = coords
                    name = str(name)
                    lat = float(lat); lon = float(lon)
                    if not (-90 <= lat <= 90 and -180 <= lon <= 180):
                        raise ValueError(f"Lat/Lon jo të vlefshme për '{name}': {lat}, {lon}")
                    spots.append((name, (lat, lon)))
                if len(spots) < 3:
                    raise ValueError("Duhet të kesh të paktën 3 pika.")
                return spots
        except Exception:
            # nëse JSON dështoi, provojmë Python literal (.txt)
            pass

        try:
            with open(path, "r", encoding="utf-8") as f:
                src = f.read()
            obj = ast.literal_eval(src)
            spots = []
            for item in obj:
                name, coords = item
                lat, lon = coords
                name = str(name)
                lat = float(lat); lon = float(lon)
                if not (-90 <= lat <= 90 and -180 <= lon <= 180):
                    raise ValueError(f"Lat/Lon jo të vlefshme për '{name}': {lat}, {lon}")
                spots.append((name, (lat, lon)))
            if len(spots) < 3:
                raise ValueError("Duhet të kesh të paktën 3 pika.")
            return spots
        except Exception:
            continue

    print(f"⚠️ S'gjeta dot '{tag}' si .json/.txt ose formati ishte i pavlefshëm. Po përdor default-in.")
    return default_spots



def _ask_spot_list_literal(default_spots):
    print(
        "👉 Zgjidh hyrjen për pikat turistike:\n"
        "   - Shtyp Enter për default\n"
        "   - Shkruaj L1, L2 ose L3 për të lexuar nga skedari përkatës në folderin aktual\n"
        "   - Shkruaj ANY për të ngjitur manualisht një listë Python\n"
        "Formati manual: [(\"Kalaja\", (41.3275, 19.8189)), (\"Muzeu Historik\", (41.3305, 19.8189)), ...]\n"
        "Për L1/L2/L3, lejohet format JSON ose Python literal (shih më poshtë)."
    )
    first = input("Zgjedhja juaj (Enter / L1 / L2 / L3 / ANY): ").strip()

    # 1) Default
    if first == "":
        return default_spots

    # 2) Lexim nga skedarët L1/L2/L3
    if first.upper() in {"L1", "L2", "L3"}:
        return _load_spots_from_file(first.upper(), default_spots)

    # 3) Ngjitje manuale (ANY)
    if first.upper() == "ANY":
        print("Paste listën këtu (mund të jetë shumë-rreshtëshe). Shkruaj 'END' për ta mbyllur:")
        buf = []
        while True:
            line = input()
            if line.strip() == "END":
                break
            buf.append(line)
        src = "\n".join(buf).strip()
        if not src:
            return default_spots
        try:
            obj = ast.literal_eval(src)
        except Exception as e:
            print(f"⚠️ Input i pavlefshëm ({e}). Po përdor default-in.")
            return default_spots

        # Validim: [(name, (lat, lon)), ...]
        spots = []
        try:
            for item in obj:
                name, coords = item
                lat, lon = coords
                name = str(name)
                lat = float(lat); lon = float(lon)
                if not (-90 <= lat <= 90 and -180 <= lon <= 180):
                    raise ValueError(f"Lat/Lon jo të vlefshme për '{name}': {lat}, {lon}")
                spots.append((name, (lat, lon)))
            if len(spots) < 3:
                raise ValueError("Duhet të kesh të paktën 3 pika.")
        except Exception as e:
            print(f"⚠️ Struktura e listës s’është e vlefshme ({e}). Po përdor default-in.")
            return default_spots

        return spots

    # 4) Çdo gjë tjetër → default
    print("⚠️ Zgjedhje e panjohur. Po përdor default-in.")
    return default_spots


if 'tourist_spots' not in globals():
    _default_spots = [
        ("Kalaja", (41.3275, 19.8189)),
        ("Muzeu Historik", (41.3305, 19.8189)),
        ("Parku i Qytetit", (41.3161, 19.8271)),
        ("Teatri Kombëtar", (41.3270, 19.8198)),
        ("Sheshi Kryesor", (41.3280, 19.8200)),
        ("Katedralja", (41.3285, 19.8208)),
        ("Pazari i Ri", (41.3308, 19.8223)),
        ("Stadiumi Air Albania", (41.3191, 19.8167)),
        ("Biblioteka Qendrore", (41.3278, 19.8197)),
        ("Galeria e Arteve", (41.3279, 19.8195))
    ]
    tourist_spots = _ask_spot_list_literal(_default_spots)

# vazhdo si më parë:
names = [spot[0] for spot in tourist_spots]
if 'positions' not in globals():
    positions = {i: spot[1] for i, spot in enumerate(tourist_spots)}
if 'num_points' not in globals():
    num_points = len(tourist_spots)


# pjesa jote vijon siç është:
names = [spot[0] for spot in tourist_spots]
if 'positions' not in globals():
    positions = {i: spot[1] for i, spot in enumerate(tourist_spots)}
if 'num_points' not in globals():
    num_points = len(tourist_spots)

names = [spot[0] for spot in tourist_spots]
if 'positions' not in globals():
    positions = {i: spot[1] for i, spot in enumerate(tourist_spots)}  # (lat, lon)
if 'num_points' not in globals():
    num_points = len(tourist_spots)

# ---- 1) Haversine and distance matrix ----
if 'haversine' not in globals():
    from math import radians, cos, sin, asin, sqrt
    def haversine(coord1, coord2):
        lat1, lon1 = coord1; lat2, lon2 = coord2
        lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
        dlat = lat2 - lat1; dlon = lon2 - lon1
        a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
        c = 2 * asin(sqrt(a)); r = 6371  # km
        return c * r * 1000  # meters

if 'distance_matrix' not in globals():
    def distance_matrix():
        m = np.zeros((num_points, num_points))
        for i in range(num_points):
            for j in range(num_points):
                m[i, j] = haversine(positions[i], positions[j])
        return m

if 'dist_matrix' not in globals():
    dist_matrix = distance_matrix()

if 'total_distance' not in globals():
    def total_distance(route):
        return sum(dist_matrix[route[i], route[(i+1) % num_points]] for i in range(num_points))

# ---- 2) Brute Force baseline (graph-free, directly on dist_matrix) ----
# (Nuk ka nevojë për networkx; logjika është identike me enumerimin e të gjitha tureve)
if 'solve_tsp_brute_force' not in globals():
    def solve_tsp_brute_force():
        nodes = list(range(num_points))
        best_path, best_dist = None, float('inf')
        for perm in itertools.permutations(nodes[1:]):  # fiks start = nodes[0]
            path = [nodes[0]] + list(perm) + [nodes[0]]
            d = sum(dist_matrix[path[i], path[i+1]] for i in range(len(path)-1))
            if d < best_dist:
                best_dist, best_path = d, path
        return best_path, best_dist
# --- NEW: Brute Force me kufi kohe 90s (soft-timeout) ---
def solve_tsp_brute_force_timed(max_seconds=90):
    nodes = list(range(num_points))
    best_path, best_dist = None, float('inf')
    t0 = time.time()
    for perm in itertools.permutations(nodes[1:]):  # fiks start = nodes[0]
        if time.time() - t0 > max_seconds:
            raise TimeoutError("Brute Force exceeded 90 seconds.")
        path = [nodes[0]] + list(perm) + [nodes[0]]
        d = sum(dist_matrix[path[i], path[i+1]] for i in range(len(path)-1))
        if d < best_dist:
            best_dist, best_path = d, path
    return best_path, best_dist, time.time() - t0

# Provo BF me timeout; nëse tejkalohet, e heqim nga konsiderata
try:
    if ('bf_route' not in globals()) or ('bf_dist' not in globals()) or ('bf_time' not in globals()):
        bf_route, bf_dist, bf_time = solve_tsp_brute_force_timed(max_seconds=90)
        print(f"[BF baseline] dist={bf_dist:.2f} m | time={bf_time:.4f} s")
except TimeoutError:
    print("⏱️ Brute Force u anashkalua: koha tejkaloi 90 sekonda.")
    bf_route, bf_dist, bf_time = None, None, None


if 'bf_dist' not in globals() or 'bf_route' not in globals() or 'bf_time' not in globals():
    t0 = time.time()
    bf_route, bf_dist = solve_tsp_brute_force()
    bf_time = time.time() - t0
    print(f"[BF baseline] dist={bf_dist:.2f} m | time={bf_time:.4f} s")

# ---- 3) GA primitives (define ONLY if missing; identical logic to your code) ----
if 'create_population' not in globals():
    def create_population(pop_size):
        return [random.sample(range(num_points), num_points) for _ in range(pop_size)]

if 'selection' not in globals():
    def selection(population):
        return sorted(population, key=lambda x: total_distance(x))[:len(population)//2]

if 'crossover' not in globals():
    def crossover(p1, p2):
        a, b = sorted(random.sample(range(num_points), 2))
        child = [-1]*num_points
        child[a:b] = p1[a:b]
        fill = [c for c in p2 if c not in child]
        i = 0
        for j in range(num_points):
            if child[j] == -1:
                child[j] = fill[i]
                i += 1
        return child

if 'mutate' not in globals():
    def mutate(route, rate):
        for i in range(num_points):
            if random.random() < rate:
                j = random.randint(0, num_points-1)
                route[i], route[j] = route[j], route[i]
        return route

# ---- 4) Instrumented GA/SA (tracked) without changing their search logic ----
def run_ga_tracked(pop_size, generations=50, mutation_rate=0.02):
    population = create_population(pop_size)
    best_route = None
    best_dist = float('inf')
    start_time = time.time()
    history = []
    for _ in range(generations):
        selected = selection(population)
        children = []
        while len(children) < pop_size:
            p1, p2 = random.sample(selected, 2)
            child = crossover(p1, p2)
            child = mutate(child, mutation_rate)
            children.append(child)
        population = children
        current_best = min(population, key=lambda x: total_distance(x))
        current_best_dist = total_distance(current_best)
        if current_best_dist < best_dist:
            best_dist = current_best_dist
            best_route = current_best
        history.append(best_dist)
    return best_route, best_dist, time.time() - start_time, history

def run_sa_tracked(temperature=10000, cooling_rate=0.995, log_every=100):
    current_route = list(range(num_points))
    random.shuffle(current_route)
    best_route = list(current_route)
    best_dist = total_distance(best_route)
    start_time = time.time()
    history = [best_dist]
    steps = 0
    T = float(temperature)
    while T > 1:
        new_route = list(current_route)
        i, j = sorted(random.sample(range(num_points), 2))
        new_route[i:j] = reversed(new_route[i:j])
        current_dist = total_distance(current_route)
        new_dist = total_distance(new_route)
        if new_dist < current_dist or random.random() < np.exp((current_dist - new_dist) / T):
            current_route = new_route
            if new_dist < best_dist:
                best_route = new_route
                best_dist = new_dist
        steps += 1
        if steps % log_every == 0:
            history.append(best_dist)
        T *= cooling_rate
    if history[-1] != best_dist:
        history.append(best_dist)
    return best_route, best_dist, time.time() - start_time, history

# ---- 5) Metrics & helpers ----
def optimality_gap(dist, best):  # in %
    return max(0.0, (dist - best) / best * 100.0)

def success(dist, best, eps=0.0):
    return 1 if (dist - best) <= eps else 0

# ---- 6) Experiment protocol ----
N_RUNS = 30
GA_PARAMS = dict(pop_size=50, generations=50, mutation_rate=0.02)
SA_PARAMS = dict(temperature=10000, cooling_rate=0.995, log_every=100)
EPSILON_METERS = 0.0  # vendos p.sh. 1.0 për tolerancë
# --- NEW: BEST_KNOWN i sigurt në mungesë të BF ---
if ('bf_dist' in globals()) and (bf_dist is not None) and np.isfinite(bf_dist):
    BEST_KNOWN = bf_dist
else:
    BEST_KNOWN = np.inf  # përkohësisht; do ta zëvendësojmë pasi të kemi GA/SA



def run_many_ga(n_runs=N_RUNS, params=GA_PARAMS):
    rows, histories = [], []
    t0 = time.time()
    for r in range(n_runs):
        seed_val = 42 + r
        random.seed(seed_val); np.random.seed(seed_val)
        _, dist, t, hist = run_ga_tracked(**params)
        rows.append(dict(run=r+1, distance=dist, time=t,
                         gap=optimality_gap(dist, BEST_KNOWN),
                         success=success(dist, BEST_KNOWN, EPSILON_METERS)))
        histories.append(hist)
        if (r+1) % 5 == 0 or (r+1) == n_runs:
            elapsed = time.time() - t0
            print(f"[GA] run {r+1}/{n_runs} | best={dist:.1f} m | time={t:.3f}s | elapsed={elapsed:.2f}s", flush=True)
    return pd.DataFrame(rows), histories

def run_many_sa(n_runs=N_RUNS, params=SA_PARAMS):
    rows, histories = [], []
    t0 = time.time()
    for r in range(n_runs):
        seed_val = 4242 + r
        random.seed(seed_val); np.random.seed(seed_val)
        _, dist, t, hist = run_sa_tracked(**params)
        rows.append(dict(run=r+1, distance=dist, time=t,
                         gap=optimality_gap(dist, BEST_KNOWN),
                         success=success(dist, BEST_KNOWN, EPSILON_METERS)))
        histories.append(hist)
        if (r+1) % 5 == 0 or (r+1) == n_runs:
            elapsed = time.time() - t0
            print(f"[SA] run {r+1}/{n_runs} | best={dist:.1f} m | time={t:.3f}s | elapsed={elapsed:.2f}s", flush=True)
    return pd.DataFrame(rows), histories


ga_df, ga_hist = run_many_ga()
sa_df, sa_hist = run_many_sa()

# Nëse BF mungon, përdor më të mirin nga GA/SA si referencë dhe përditëso gap/success
if (('bf_dist' not in globals()) or (bf_dist is None) or (not np.isfinite(BEST_KNOWN))):
    _best_from_meta = min(ga_df['distance'].min(), sa_df['distance'].min())
    BEST_KNOWN = _best_from_meta  # vendose baseline-in

    # Rillogarit kolonat që varen nga BEST_KNOWN
    for _df in (ga_df, sa_df):
        _df['gap'] = ((_df['distance'] - _best_from_meta) / _best_from_meta * 100.0).clip(lower=0.0)
        _df['success'] = (_df['distance'] - _best_from_meta <= EPSILON_METERS).astype(int)


# ---- 7) Summary table (mean ± std) ----
def summarize(df, algo_name):
    return {
        "Algorithm": algo_name,
        "N": num_points,
        "Distance_mean": df["distance"].mean(),
        "Distance_std": df["distance"].std(ddof=1),
        "Time_mean": df["time"].mean(),
        "Time_std": df["time"].std(ddof=1),
        "Success_rate_%": 100.0 * df["success"].mean(),
        "Gap_mean_%": df["gap"].mean(),
        "Gap_std_%": df["gap"].std(ddof=1),
    }


# --- NEW: Hiq BF nga summary nëse u anashkalua; përndryshe lëre si më parë ---
if (('bf_dist' not in globals()) or (bf_dist is None)):
    summary = [
        summarize(ga_df, "Genetic Algorithm"),
        summarize(sa_df, "Simulated Annealing"),
    ]
else:
    summary = [
        {
            "Algorithm": "Brute Force",
            "N": num_points,
            "Distance_mean": bf_dist,
            "Distance_std": 0.0,
            "Time_mean": bf_time,
            "Time_std": 0.0,
            "Success_rate_%": 100.0,
            "Gap_mean_%": 0.0,
            "Gap_std_%": 0.0,
        },
        summarize(ga_df, "Genetic Algorithm"),
        summarize(sa_df, "Simulated Annealing"),
    ]

summary_df = pd.DataFrame(summary)

# ---- 8) Save outputs ----
os.makedirs("reports", exist_ok=True)
ga_df.to_csv(f"reports/runs_ga.csv", index=False)
sa_df.to_csv(f"reports/runs_sa.csv", index=False)
summary_df.to_csv(f"reports/summary_n{num_points}_metrics.csv", index=False)

# Boxplots
plt.figure(figsize=(6,4))
plt.boxplot([ga_df["distance"], sa_df["distance"]], labels=["GA", "SA"], showmeans=True)
plt.ylabel("Distance (m)")
plt.title(f"Distance distribution over runs (N={num_points})")
plt.tight_layout()
plt.savefig(f"reports/boxplot_distance_n{num_points}.png", dpi=600, bbox_inches="tight")
plt.close()

plt.figure(figsize=(6,4))
plt.boxplot([ga_df["time"], sa_df["time"]], labels=["GA", "SA"], showmeans=True)
plt.ylabel("Time (s)")
plt.title(f"Runtime distribution over runs (N={num_points})")
plt.tight_layout()
plt.savefig(f"reports/boxplot_time_n{num_points}.png", dpi=600, bbox_inches="tight")
plt.close()

# Convergence (median + IQR)
def plot_convergence(histories, title, outpath):
    min_len = min(len(h) for h in histories)
    H = np.array([h[:min_len] for h in histories])
    med = np.median(H, axis=0)
    q1 = np.percentile(H, 25, axis=0)
    q3 = np.percentile(H, 75, axis=0)
    plt.figure(figsize=(7,4))
    x = np.arange(min_len)
    plt.plot(x, med, linewidth=2)
    plt.fill_between(x, q1, q3, alpha=0.2)
    plt.xlabel("Iteration")
    plt.ylabel("Best-so-far distance (m)")
    plt.title(f"{title} convergence (median & IQR) — N={num_points}")
    plt.tight_layout()
    plt.savefig(outpath, dpi=600, bbox_inches="tight")
    plt.close()

plot_convergence(ga_hist, "GA", f"reports/convergence_ga_n{num_points}.png")
plot_convergence(sa_hist, "SA", f"reports/convergence_sa_n{num_points}.png")

# ---- 9) Print summary & file list ----
print(f"\n=== Summary (N={num_points}) ===")
print(summary_df.to_string(index=False, formatters={
    "Distance_mean": lambda v: f"{v:.2f}",
    "Distance_std":  lambda v: f"{v:.2f}",
    "Time_mean":     lambda v: f"{v:.4f}",
    "Time_std":      lambda v: f"{v:.4f}",
    "Success_rate_%":lambda v: f"{v:.1f}",
    "Gap_mean_%":    lambda v: f"{v:.3f}",
    "Gap_std_%":     lambda v: f"{v:.3f}",
}))
print("\nFiles saved in ./reports/:")
for f in sorted(os.listdir("reports")):
    print(" -", f)

# Ruaj spotet për përdorim nga qelizat e tjera
import os, json
os.makedirs("reports", exist_ok=True)
with open("reports/tourist_spots.json", "w", encoding="utf-8") as f:
    json.dump(tourist_spots, f, ensure_ascii=False)
print("💾 Saved spots to reports/tourist_spots.json")
# ==========================================================================================



👉 Zgjidh hyrjen për pikat turistike:
   - Shtyp Enter për default
   - Shkruaj L1, L2 ose L3 për të lexuar nga skedari përkatës në folderin aktual
   - Shkruaj ANY për të ngjitur manualisht një listë Python
Formati manual: [("Kalaja", (41.3275, 19.8189)), ("Muzeu Historik", (41.3305, 19.8189)), ...]
Për L1/L2/L3, lejohet format JSON ose Python literal (shih më poshtë).


Zgjedhja juaj (Enter / L1 / L2 / L3 / ANY):  L3


⏱️ Brute Force u anashkalua: koha tejkaloi 90 sekonda.


C:\Users\Alketa\AppData\Local\Temp\ipykernel_16076\1900024258.py:324: RuntimeWarning: invalid value encountered in scalar divide
  return max(0.0, (dist - best) / best * 100.0)


[GA] run 5/30 | best=10510.7 m | time=0.274s | elapsed=1.30s
[GA] run 10/30 | best=11067.0 m | time=0.220s | elapsed=2.56s
[GA] run 15/30 | best=8962.1 m | time=0.270s | elapsed=3.80s
[GA] run 20/30 | best=10509.6 m | time=0.231s | elapsed=4.93s
[GA] run 25/30 | best=8271.2 m | time=0.254s | elapsed=6.24s
[GA] run 30/30 | best=9838.4 m | time=0.268s | elapsed=7.52s
[SA] run 5/30 | best=6415.1 m | time=0.126s | elapsed=0.63s
[SA] run 10/30 | best=7521.3 m | time=0.145s | elapsed=1.27s
[SA] run 15/30 | best=6387.2 m | time=0.128s | elapsed=1.90s
[SA] run 20/30 | best=6407.1 m | time=0.147s | elapsed=2.57s
[SA] run 25/30 | best=6713.2 m | time=0.129s | elapsed=3.19s
[SA] run 30/30 | best=6499.3 m | time=0.127s | elapsed=3.81s

=== Summary (N=31) ===
          Algorithm  N Distance_mean Distance_std Time_mean Time_std Success_rate_% Gap_mean_% Gap_std_%
  Genetic Algorithm 31      10143.11       915.95    0.2478   0.0268            0.0     62.791    14.700
Simulated Annealing 31       6782

In [2]:
# Auto-load nëse tourist_spots mungon
import os, json
if 'tourist_spots' not in globals():
    path = "reports/tourist_spots.json"
    if os.path.exists(path):
        with open(path, "r", encoding="utf-8") as f:
            ts = json.load(f)
        # konverto strukturën JSON -> lista tuples [(name, (lat,lon)), ...]
        tourist_spots = [(name, tuple(coords)) for name, coords in ts]
        print(f"📥 Loaded {len(tourist_spots)} spots from {path}")
    else:
        raise RuntimeError("tourist_spots mungon. Ekzekuto qelizën e INPUT/GEOCODE ose siguro JSON-in.")

# ==================== 11) Route figures (itineraries) — 600 dpi, saved ====================
import os, time, itertools, random
import numpy as np

os.makedirs("reports", exist_ok=True)


if 'tourist_spots' not in globals():
    raise RuntimeError("tourist_spots nuk u gjet. Ekzekuto qelizën e INPUT/GEOCODE fillimisht.")

# Derivo nga tourist_spots (pa fallback liste fikse)
names = [s[0] for s in tourist_spots]
positions = {i: s[1] for i, s in enumerate(tourist_spots)}
num_points = len(tourist_spots)

# Haversine, dist_matrix, total_distance (defino nëse mungojnë)
if 'haversine' not in globals():
    from math import radians, cos, sin, asin, sqrt
    def haversine(coord1, coord2):
        lat1, lon1 = coord1; lat2, lon2 = coord2
        lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
        dlat = lat2 - lat1; dlon = lon2 - lon1
        a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
        c = 2 * asin(sqrt(a)); r = 6371
        return c * r * 1000  # m

def distance_matrix():
    m = np.zeros((num_points, num_points))
    for i in range(num_points):
        for j in range(num_points):
            m[i, j] = haversine(positions[i], positions[j])
    return m

dist_matrix = distance_matrix()

def total_distance(route):
    return sum(dist_matrix[route[i], route[(i+1) % num_points]] for i in range(num_points))

# Brute force mbi matricë
def _bf_on_matrix():
    nodes = list(range(num_points))
    best_path, best_dist = None, float('inf')
    for perm in itertools.permutations(nodes[1:]):
        path = [nodes[0]] + list(perm)  # mbyllja bëhet nga total_distance me modulo
        d = total_distance(path)
        if d < best_dist:
            best_dist, best_path = d, path
    return best_path, best_dist

# --- NEW: Brute Force me kufi kohe 90s (soft timeout) ---
def solve_tsp_brute_force_timed(max_seconds=90):
    nodes = list(range(num_points))
    best_path, best_dist = None, float('inf')
    t0 = time.time()
    for perm in itertools.permutations(nodes[1:]):  # fiks start = nodes[0]
        # Ndal pas 90 sekondash
        if time.time() - t0 > max_seconds:
            raise TimeoutError("Brute Force exceeded 90 seconds.")
        path = [nodes[0]] + list(perm) + [nodes[0]]
        d = sum(dist_matrix[path[i], path[i+1]] for i in range(len(path)-1))
        if d < best_dist:
            best_dist, best_path = d, path
    return best_path, best_dist, time.time() - t0

# Provo BF me timeout; nëse tejkalohet, e heqim nga konsiderata
try:
    if ('bf_route' not in globals()) or ('bf_dist' not in globals()) or ('bf_time' not in globals()):
        bf_route, bf_dist, bf_time = solve_tsp_brute_force_timed(max_seconds=90)
        print(f"[BF baseline] dist={bf_dist:.2f} m | time={bf_time:.4f} s")
except TimeoutError as _e:
    print("⏱️ Brute Force u anashkalua: koha tejkaloi 90 sekonda.")
    bf_route, bf_dist, bf_time = None, None, None


if 'bf_route' not in globals() or 'bf_dist' not in globals() or 'bf_time' not in globals():
    t0 = time.time()
    bf_route, bf_dist = _bf_on_matrix()
    bf_time = time.time() - t0

def _get_ga_route():
    if 'run_ga_tracked' in globals():
        random.seed(123); np.random.seed(123)
        r, d, t, _ = run_ga_tracked(pop_size=50, generations=50, mutation_rate=0.02)
        return r, d, t
    elif 'run_ga' in globals():
        random.seed(123); np.random.seed(123)
        return run_ga(pop_size=50, generations=50, mutation_rate=0.02)
    else:
        raise RuntimeError("GA functions not found (run_ga_tracked/run_ga).")

def _get_sa_route():
    if 'run_sa_tracked' in globals():
        random.seed(321); np.random.seed(321)
        r, d, t, _ = run_sa_tracked(temperature=10000, cooling_rate=0.995, log_every=100)
        return r, d, t
    elif 'run_sa' in globals():
        random.seed(321); np.random.seed(321)
        return run_sa(temperature=10000, cooling_rate=0.995)
    else:
        raise RuntimeError("SA functions not found (run_sa_tracked/run_sa).")
        
ga_route, ga_dist, ga_time = _get_ga_route()
sa_route, sa_dist, sa_time = _get_sa_route()


# Funksioni i ri i vizualizimit me dpi=600 (pa prekur plot_on_map ekzistues)
def plot_itinerary_600dpi(route, title, dist, time_sec, color, outpath):
    import geopandas as gpd
    from shapely.geometry import Point, LineString
    import contextily as ctx
    import matplotlib.pyplot as plt
    from matplotlib.gridspec import GridSpec

    coords = [positions[i][::-1] for i in route] + [positions[route[0]][::-1]]
    gdf_points = gpd.GeoDataFrame(geometry=[Point(xy) for xy in coords], crs="EPSG:4326").to_crs(epsg=3857)
    gdf_line = gpd.GeoDataFrame(geometry=[LineString([p for p in gdf_points.geometry])], crs="EPSG:3857")

    fig = plt.figure(figsize=(10, 7))
    gs = GridSpec(1, 2, width_ratios=[3, 1])

    # Harta
    ax_map = fig.add_subplot(gs[0])
    gdf_line.plot(ax=ax_map, color=color, linewidth=2, label=title)
    gdf_points.plot(ax=ax_map, color='red')
    for i, point in enumerate(gdf_points.geometry[:-1]):
        ax_map.text(point.x + 5, point.y + 5, f"{i+1}", fontsize=9)
    try:
        ctx.add_basemap(ax_map, source=ctx.providers.OpenStreetMap.Mapnik)
    except Exception as e:
        print("Basemap skipped:", e)
    ax_map.set_title(f"{title}\nDistance: {dist:.0f} m | Time: {time_sec:.2f} s", fontsize=12)
    ax_map.axis('off')

    # Itinerari (tekst)
    ax_txt = fig.add_subplot(gs[1])
    ax_txt.axis('off')
    ax_txt.set_title("Itinerary", fontsize=12, pad=20)
    itin = [f"{i+1}. {names[node]}" for i, node in enumerate(route)]
    itin.append(f"{len(route)+1}. {names[route[0]]} (kthim)")
    ax_txt.text(0, 1, "\n".join(itin), fontsize=9, va='top', family='monospace')

    plt.tight_layout()
    plt.savefig(outpath, dpi=600, bbox_inches="tight")
    plt.close()
    print("Saved:", outpath)

os.makedirs("reports", exist_ok=True)
# Gjenero dhe ruaj imazhet (PNG, 600 dpi)
# --- NEW: Mos vizato BF nëse është anashkaluar ---
if (('bf_route' in globals()) and (bf_route is not None) and (bf_dist is not None)):
    plot_itinerary_600dpi(bf_route, "Brute Force", bf_dist, bf_time, color='green',
                          outpath=f"reports/route_bruteforce_n{num_points}.png")
else:
    print("BF route figure skipped (timeout).")


plot_itinerary_600dpi(ga_route, "Genetic Algorithm", ga_dist, ga_time, color='blue',
                      outpath=f"reports/route_ga_n{num_points}.png")
plot_itinerary_600dpi(sa_route, "Simulated Annealing", sa_dist, sa_time, color='orange',
                      outpath=f"reports/route_sa_n{num_points}.png")




BF route figure skipped (timeout).
Saved: reports/route_ga_n31.png
Saved: reports/route_sa_n31.png
